In [ ]:
# ===========================
# JUPYTER CELL: Launch Best_SSL_Model_Selection.py (run inside notebook)
# ===========================
import os, sys, subprocess, pathlib, glob, os.path as op

PROJECT_DIR = "/work/projects/myproj/Linear_Probing_For_SSL"
SCRIPT = "Best_SSL_Model_Selection.py"
REPO_ROOT = str(pathlib.Path(PROJECT_DIR).resolve().parent)

def _detect_user_base():
    aau = glob.glob("/work/Member Files:*")
    if aau:
        return op.basename(aau[0])
    sdu = [d for d in glob.glob("/work/*#*") if op.isdir(d)]
    return op.basename(sdu[0]) if sdu else ""

USER_BASE_DIR = os.environ.get("USER_BASE_DIR") or _detect_user_base() or ""
if USER_BASE_DIR:
    os.environ["USER_BASE_DIR"] = USER_BASE_DIR
WORK_ROOT = pathlib.Path("/work") / USER_BASE_DIR if USER_BASE_DIR else pathlib.Path.cwd()

# Which classes to probe:
# "leu"  -> only Leucocyte
# "epi"  -> only Squamous Epithelial Cell
# "all"  -> both
os.environ["RFDETR_PROBE_TARGET"] = "epi"

# Probe settings
os.environ["RFDETR_USE_PATCH_224"] = "1"
os.environ["RFDETR_PATCH_SIZE"] = "224"
os.environ["RFDETR_TRAIN_FRACTION"] = "0.125"
os.environ["RFDETR_FRACTION_SEED"] = "42"
os.environ["SEED"] = "42"
os.environ.setdefault("NUM_WORKERS", "8")

# Required paths (mounted-root friendly defaults)
os.environ.setdefault("SSL_CKPT_ROOT", str(WORK_ROOT / "SSL_Checkpoints"))
os.environ.setdefault("STAT_DATASETS_ROOT", "/work/projects/myproj/SOLO_Supervised_RFDETR/Stat_Dataset")
os.environ.setdefault("DATASET_PREFIX_LEU", "QA-2025v2_Leucocyte_OVR")
os.environ.setdefault("DATASET_PREFIX_EPI", "QA-2025v2_SquamousEpithelialCell_OVR")

# Optional explicit dataset overrides (if set, they override STAT_DATASETS_ROOT+prefix)
# os.environ["DATASET_LEUCO_DIR"] = "/work/projects/myproj/SOLO_Supervised_RFDETR/Stat_Dataset/QA-2025v2_Leucocyte_OVR_..."
# os.environ["DATASET_EPI_DIR"] = "/work/projects/myproj/SOLO_Supervised_RFDETR/Stat_Dataset/QA-2025v2_SquamousEpithelialCell_OVR_..."

# Where outputs are written: script creates session_YYYYMMDD_HHMMSS under this
os.environ.setdefault("OUTPUT_BASE", "/work/projects/myproj/Linear_Probing_For_SSL/SSL_SELECTION")

# Optional: explicit checkpoint list (comma-separated). If unset, script scans SSL_CKPT_ROOT.
# os.environ["SSL_CKPTS"] = "epoch_epoch-004.ckpt,epoch_epoch-009.ckpt,epoch_epoch-014.ckpt,epoch_epoch-029.ckpt,last.ckpt"

# Optional image root used for resolving file_name entries in COCO json
# os.environ["IMAGES_FALLBACK_ROOT"] = "/work/Member Files:yourname/CellScanData/Zoom10x - Quality Assessment_Cleaned"

# CUDA allocator safety
os.environ.setdefault("PYTORCH_CUDA_ALLOC_CONF", "expandable_segments:True")
os.environ.setdefault("TF32", "1")

wd = pathlib.Path(PROJECT_DIR).resolve()
py = sys.executable
cmd = [py, "-u", str(wd / SCRIPT)]

print("\n[LAUNCH]")
print(" cwd:", wd)
print(" cmd:", cmd)
print(" env: USER_BASE_DIR=", os.environ.get("USER_BASE_DIR"))
print(" env: WORK_ROOT=", WORK_ROOT)
print(" env: RFDETR_PROBE_TARGET=", os.environ.get("RFDETR_PROBE_TARGET"))
print(" env: SSL_CKPT_ROOT=", os.environ.get("SSL_CKPT_ROOT"))
print(" env: STAT_DATASETS_ROOT=", os.environ.get("STAT_DATASETS_ROOT"))
print(" env: DATASET_PREFIX_LEU=", os.environ.get("DATASET_PREFIX_LEU"))
print(" env: DATASET_PREFIX_EPI=", os.environ.get("DATASET_PREFIX_EPI"))
print(" env: RFDETR_TRAIN_FRACTION=", os.environ.get("RFDETR_TRAIN_FRACTION"))
print(" env: RFDETR_FRACTION_SEED=", os.environ.get("RFDETR_FRACTION_SEED"))
print(" env: OUTPUT_BASE=", os.environ.get("OUTPUT_BASE"))

# Stream output live so training progress is visible in-notebook
print("\n========== LIVE OUTPUT ==========")
p = subprocess.Popen(
    cmd,
    cwd=str(wd),
    text=True,
    bufsize=1,
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
)
for line in p.stdout:
    print(line, end="")
p.wait()
print("\n[RETURNCODE]", p.returncode)
if p.returncode != 0:
    raise subprocess.CalledProcessError(p.returncode, cmd)
